# Types Operable - Structured LLM Output Collections

The `types.operable` module provides validated Spec collections for generating structured LLM outputs:

**Core Component:**
- **Operable**: Ordered, validated collection of Specs with name uniqueness enforcement

**Key Features:**
- Framework-agnostic Spec collections (layer 2 of 3-layer architecture)
- Field filtering (include/exclude for partial models)
- Model generation via adapters (Pydantic, attrs, dataclasses)
- Fuzzy field matching for flexible LLM outputs
- Integration with Spec for validation and type safety

**Architecture:**
- **Layer 1 (Spec)**: Individual field schema
- **Layer 2 (Operable)**: Validated Spec collection ← This module
- **Layer 3 (Adapter)**: Framework-specific model generation

In [1]:
from lionpride.types import Operable, Spec, Unset
from lionpride.types.spec_adapters.pydantic_field import PydanticSpecAdapter

## 1. Basic Operable Creation

Operable manages collections of Specs with validation and ordering preservation.

In [2]:
# Create Specs for a user model
username_spec = Spec(str, name="username", description="User's username")
email_spec = Spec(str, name="email", description="User's email address")
age_spec = Spec(int, name="age", nullable=True, description="User's age")

# Create Operable from Specs
user_operable = Operable((username_spec, email_spec, age_spec), name="User")

print(f"Operable name: {user_operable.name}")
print(f"Number of fields: {len(user_operable.__op_fields__)}")
print(f"Allowed fields: {user_operable.allowed()}")

Operable name: User
Number of fields: 3
Allowed fields: {'email', 'username', 'age'}


In [3]:
# Operable accepts lists (converts to tuple internally)
specs_list = [Spec(str, name="field1"), Spec(int, name="field2"), Spec(bool, name="field3")]

operable_from_list = Operable(specs_list, name="Example")
print(f"Type of __op_fields__: {type(operable_from_list.__op_fields__)}")
print(f"Fields are immutable: {isinstance(operable_from_list.__op_fields__, tuple)}")

Type of __op_fields__: <class 'tuple'>
Fields are immutable: True


In [4]:
# Empty Operable is valid
empty_operable = Operable()
print(f"Empty operable allowed fields: {empty_operable.allowed()}")
print(f"Length: {len(empty_operable.__op_fields__)}")

Empty operable allowed fields: set()
Length: 0


## 2. Name Uniqueness Validation

Operable enforces unique field names at construction time (fail-fast principle).

In [5]:
# Duplicate names raise ValueError
spec1 = Spec(str, name="field1")
spec2 = Spec(int, name="field1")  # Duplicate name!

try:
    invalid_operable = Operable((spec1, spec2))
except ValueError as e:
    print(f"✓ Duplicate detection: {e}")

✓ Duplicate detection: Duplicate field names found: ['field1']. Each spec must have a unique name.


In [6]:
# Type validation - only Spec objects allowed
try:
    invalid_operable = Operable(("not_a_spec", spec1))
except TypeError as e:
    print(f"✓ Type validation: {e}")

✓ Type validation: All specs must be Spec objects, got str at index 0


## 3. Field Access and Validation

Operable provides methods to query and validate field membership.

In [7]:
# allowed() returns set of field names
print(f"Allowed fields: {user_operable.allowed()}")
print(f"Type: {type(user_operable.allowed())}")

# O(1) membership check
print(f"\n'username' allowed: {'username' in user_operable.allowed()}")
print(f"'password' allowed: {'password' in user_operable.allowed()}")

Allowed fields: {'email', 'username', 'age'}
Type: <class 'set'>

'username' allowed: True
'password' allowed: False


In [8]:
# check_allowed() validates field names (default: raise on error)
result = user_operable.check_allowed("username")
print(f"Valid field check: {result}")

# Invalid field raises ValueError
try:
    user_operable.check_allowed("invalid_field")
except ValueError as e:
    print(f"\n✓ Invalid field rejected: {e}")

Valid field check: True

✓ Invalid field rejected: Some specified fields are not allowed: {'invalid_field'}


In [9]:
# check_allowed() with as_boolean=True returns bool (no exception)
is_valid = user_operable.check_allowed("username", as_boolean=True)
is_invalid = user_operable.check_allowed("password", as_boolean=True)

print(f"'username' exists: {is_valid}")
print(f"'password' exists: {is_invalid}")

# Useful for conditional logic
if user_operable.check_allowed("email", as_boolean=True):
    print("\n✓ Email field is available")

'username' exists: True
'password' exists: False

✓ Email field is available


In [10]:
# get() retrieves Spec by name
username_retrieved = user_operable.get("username")
print(f"Retrieved spec: {username_retrieved}")
print(f"Is same object: {username_retrieved is username_spec}")

# Missing field returns Unset (not None)
missing = user_operable.get("password")
print(f"\nMissing field returns: {missing}")
print(f"Is Unset: {missing is Unset}")

# Can provide custom default
default_spec = user_operable.get("password", default="custom_default")
print(f"With custom default: {default_spec}")

Retrieved spec: Spec(base_type=<class 'str'>, metadata=(Meta(key='name', value='username'), Meta(key='description', value="User's username")))
Is same object: True

Missing field returns: Unset
Is Unset: True
With custom default: custom_default


## 4. Field Filtering - Include/Exclude

Generate partial models from the same Operable (DRY principle).

In [11]:
# Create Operable with multiple fields
full_specs = [
    Spec(str, name="username"),
    Spec(str, name="email"),
    Spec(str, name="password"),
    Spec(str, name="internal_id"),
    Spec(bool, name="is_active"),
]

full_operable = Operable(full_specs, name="FullUser")
print(f"Full fields: {full_operable.allowed()}")

Full fields: {'email', 'is_active', 'internal_id', 'username', 'password'}


In [12]:
# get_specs() with include - whitelist pattern
public_specs = full_operable.get_specs(include={"username", "email", "is_active"})
print(f"Public fields (include): {[s.name for s in public_specs]}")
print(f"Type: {type(public_specs)}")

Public fields (include): ['email', 'is_active', 'username']
Type: <class 'tuple'>


In [13]:
# get_specs() with exclude - blacklist pattern
safe_specs = full_operable.get_specs(exclude={"password", "internal_id"})
print(f"Safe fields (exclude): {[s.name for s in safe_specs]}")

Safe fields (exclude): ['username', 'email', 'is_active']


In [14]:
# Cannot specify both include and exclude
try:
    full_operable.get_specs(include={"username"}, exclude={"password"})
except ValueError as e:
    print(f"✓ Mutual exclusivity enforced: {e}")

✓ Mutual exclusivity enforced: Cannot specify both include and exclude


In [15]:
# Invalid field names in include/exclude raise errors
try:
    full_operable.get_specs(include={"username", "invalid_field"})
except ValueError as e:
    print(f"✓ Typo protection: {e}")

✓ Typo protection: Some specified fields are not allowed: {'invalid_field'}


## 5. Creating Pydantic Models

Operable delegates to adapters for framework-specific model generation.

In [16]:
# Create Pydantic model from Operable
user_specs = [
    Spec(str, name="username", description="User's login name"),
    Spec(str, name="email", description="Email address"),
    Spec(int, name="age", nullable=True, default=None),
]

user_op = Operable(user_specs, name="User")

# Generate Pydantic BaseModel
UserModel = user_op.create_model(adapter="pydantic")

print(f"Model name: {UserModel.__name__}")
print(f"Model fields: {list(UserModel.model_fields.keys())}")

# Can instantiate model
user = UserModel(username="alice", email="alice@example.com", age=30)
print(f"\nCreated instance: {user}")
print(f"JSON: {user.model_dump_json()}")

Model name: User
Model fields: ['username', 'email', 'age']

Created instance: username='alice' email='alice@example.com' age=30
JSON: {"username":"alice","email":"alice@example.com","age":30}


In [17]:
# Custom model name (overrides Operable name)
CustomModel = user_op.create_model(adapter="pydantic", model_name="CustomUserModel")
print(f"Custom name: {CustomModel.__name__}")

# Without Operable name, defaults to "DynamicModel"
unnamed_op = Operable([Spec(str, name="field1")])
DynamicModel = unnamed_op.create_model(adapter="pydantic")
print(f"Default name: {DynamicModel.__name__}")

Custom name: CustomUserModel
Default name: DynamicModel


In [18]:
# Create model with field filtering
full_user_specs = [
    Spec(str, name="username"),
    Spec(str, name="email"),
    Spec(str, name="password"),
    Spec(str, name="internal_id"),
]

full_user_op = Operable(full_user_specs, name="FullUser")

# Public API model - exclude sensitive fields
PublicUser = full_user_op.create_model(
    adapter="pydantic", model_name="PublicUser", exclude={"password", "internal_id"}
)

print(f"Public model fields: {list(PublicUser.model_fields.keys())}")

# Internal model - all fields
InternalUser = full_user_op.create_model(adapter="pydantic", model_name="InternalUser")

print(f"Internal model fields: {list(InternalUser.model_fields.keys())}")

Public model fields: ['username', 'email']
Internal model fields: ['username', 'email', 'password', 'internal_id']


## 6. Fuzzy Field Matching for LLM Outputs

LLMs often generate fields with slight variations. Fuzzy matching handles this gracefully.

In [19]:
# Create model expecting specific fields
task_specs = [
    Spec(str, name="task_name"),
    Spec(str, name="task_description"),
    Spec(int, name="priority"),
]

task_op = Operable(task_specs, name="Task")
TaskModel = task_op.create_model(adapter="pydantic")

print(f"Expected fields: {list(TaskModel.model_fields.keys())}")

Expected fields: ['task_name', 'task_description', 'priority']


In [20]:
# LLM output with variations (camelCase, extra spaces, underscores)
llm_output = {
    "taskName": "Implement feature",  # camelCase instead of snake_case
    "task description": "Add user auth",  # space instead of underscore
    "Priority": 1,  # capitalized
    "extra_field": "ignored",  # unknown field
}

# Fuzzy match (lenient mode) - matches variations
matched = PydanticSpecAdapter.fuzzy_match_fields(llm_output, TaskModel, strict=False)
print(f"\nMatched fields: {matched}")

# Create model instance from matched data
task = TaskModel(**matched)
print(f"\nCreated task: {task}")


Matched fields: {'task_name': 'Implement feature', 'task_description': 'Add user auth', 'extra_field': 'ignored', 'priority': 1}

Created task: task_name='Implement feature' task_description='Add user auth' priority=1


In [21]:
# Strict mode - raises on unmatched keys
llm_output_strict = {
    "task_name": "Test task",
    "task_description": "Test description",
    "priority": 2,
    "unknown_field": "value",  # This will cause error in strict mode
}

try:
    matched_strict = PydanticSpecAdapter.fuzzy_match_fields(
        llm_output_strict, TaskModel, strict=True
    )
except ValueError as e:
    print(f"✓ Strict mode error: {e}")

✓ Strict mode error: Unmatched keys found: {'unknown_field'}


## 7. Nested Models and Listable Fields

Operable supports complex nested structures via Spec modifiers.

In [22]:
# Create nested model for address
address_specs = [
    Spec(str, name="street"),
    Spec(str, name="city"),
    Spec(str, name="country"),
]

address_op = Operable(address_specs, name="Address")
AddressModel = address_op.create_model(adapter="pydantic")

# Create user model with nested address
user_with_nested_specs = [
    Spec(str, name="name"),
    Spec(AddressModel, name="address", nullable=True),  # Nested model
    Spec(str, name="tags").as_listable(),  # List of strings
]

user_nested_op = Operable(user_with_nested_specs, name="UserWithAddress")
UserNestedModel = user_nested_op.create_model(adapter="pydantic")

print(f"Model fields: {list(UserNestedModel.model_fields.keys())}")
print("\nField annotations:")
for name, field in UserNestedModel.model_fields.items():
    print(f"  {name}: {field.annotation}")

Model fields: ['name', 'address', 'tags']

Field annotations:
  name: <class 'str'>
  address: lionpride.types.spec_adapters.pydantic_field.Address | None
  tags: list[str]


In [23]:
# Create instance with nested data
user_nested = UserNestedModel(
    name="Alice",
    address={"street": "123 Main St", "city": "NYC", "country": "USA"},
    tags=["developer", "python", "ai"],
)

print(f"User: {user_nested}")
print(f"\nAddress type: {type(user_nested.address)}")
print(f"Tags type: {type(user_nested.tags)}")
print("\nJSON output:")
import json

print(json.dumps(user_nested.model_dump(), indent=2))

User: name='Alice' address=Address(street='123 Main St', city='NYC', country='USA') tags=['developer', 'python', 'ai']

Address type: <class 'lionpride.types.spec_adapters.pydantic_field.Address'>
Tags type: <class 'list'>

JSON output:
{
  "name": "Alice",
  "address": {
    "street": "123 Main St",
    "city": "NYC",
    "country": "USA"
  },
  "tags": [
    "developer",
    "python",
    "ai"
  ]
}


## 8. Validation and Defaults

Specs support validators and default values that transfer to generated models.

In [24]:
# Validator function
def validate_username(value: str) -> str:
    if len(value) < 3:
        raise ValueError("Username must be at least 3 characters")
    if not value.isalnum():
        raise ValueError("Username must be alphanumeric")
    return value.lower()  # Normalize to lowercase


# Create specs with validators and defaults
validated_specs = [
    Spec(str, name="username", validator=validate_username),
    Spec(str, name="email"),
    Spec(str, name="status", default="active"),  # Default value
    Spec(list, name="tags", default_factory=list),  # Default factory
]

validated_op = Operable(validated_specs, name="ValidatedUser")
ValidatedUserModel = validated_op.create_model(adapter="pydantic")

# Valid username
user1 = ValidatedUserModel(username="Alice123", email="alice@example.com")
print(f"Valid user: {user1}")
print(f"Username normalized: {user1.username}")
print(f"Default status: {user1.status}")
print(f"Default tags: {user1.tags}")

Valid user: username='alice123' email='alice@example.com' status='active' tags=[]
Username normalized: alice123
Default status: active
Default tags: []


In [25]:
# Invalid username (too short)
try:
    invalid_user = ValidatedUserModel(username="ab", email="test@example.com")
except Exception as e:
    print(f"✓ Validation error: {e}")

✓ Validation error: 1 validation error for ValidatedUser
username
  Value error, Username must be at least 3 characters [type=value_error, input_value='ab', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


In [26]:
# Invalid username (non-alphanumeric)
try:
    invalid_user = ValidatedUserModel(username="user@123", email="test@example.com")
except Exception as e:
    print(f"✓ Validation error: {e}")

✓ Validation error: 1 validation error for ValidatedUser
username
  Value error, Username must be alphanumeric [type=value_error, input_value='user@123', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


## 9. Practical Example - Structured LLM Output

Real-world example: Parse LLM task extraction with validation and fuzzy matching.

In [27]:
# Define schema for task extraction
task_extraction_specs = [
    Spec(str, name="task_title", description="Brief task title"),
    Spec(str, name="task_description", description="Detailed description"),
    Spec(int, name="estimated_hours", nullable=True, default=None),
    Spec(str, name="priority", default="medium"),
    Spec(str, name="assigned_to", nullable=True, default=None),
    Spec(str, name="dependencies").as_listable().as_nullable(),  # List[str] | None
]

task_extraction_op = Operable(task_extraction_specs, name="TaskExtraction")
TaskExtractionModel = task_extraction_op.create_model(adapter="pydantic")

print(f"Task extraction model: {TaskExtractionModel.__name__}")
print(f"Fields: {list(TaskExtractionModel.model_fields.keys())}")

Task extraction model: TaskExtraction
Fields: ['task_title', 'task_description', 'estimated_hours', 'priority', 'assigned_to', 'dependencies']


In [28]:
# Simulated LLM output (with variations and extra fields)
llm_task_output = {
    "TaskTitle": "Implement user authentication",  # Different case
    "task description": "Add JWT-based authentication with refresh tokens",  # Spaces
    "estimatedHours": 8,  # camelCase
    "Priority": "high",  # Capitalized
    "Dependencies": ["Setup database", "Create user model"],  # Capitalized
    "reasoning": "This is critical for security",  # Extra field (ignored)
}

# Fuzzy match and create task
matched_task_data = PydanticSpecAdapter.fuzzy_match_fields(
    llm_task_output, TaskExtractionModel, strict=False
)

print(f"\nMatched data: {matched_task_data}")

task = TaskExtractionModel(**matched_task_data)
print("\nExtracted task:")
print(json.dumps(task.model_dump(), indent=2))


Matched data: {'task_title': 'Implement user authentication', 'task_description': 'Add JWT-based authentication with refresh tokens', 'dependencies': ['Setup database', 'Create user model'], 'reasoning': 'This is critical for security', 'estimated_hours': 8, 'priority': 'high'}

Extracted task:
{
  "task_title": "Implement user authentication",
  "task_description": "Add JWT-based authentication with refresh tokens",
  "estimated_hours": 8,
  "priority": "high",
  "assigned_to": null,
  "dependencies": [
    "Setup database",
    "Create user model"
  ]
}


In [29]:
# Create public API view (exclude internal fields)
public_task_specs = task_extraction_op.get_specs(exclude={"assigned_to", "dependencies"})

public_task_op = Operable(public_task_specs, name="PublicTask")
PublicTaskModel = public_task_op.create_model(adapter="pydantic")

print(f"Public API fields: {list(PublicTaskModel.model_fields.keys())}")

# Convert full task to public view
public_task_data = task.model_dump(include=set(PublicTaskModel.model_fields.keys()))
public_task = PublicTaskModel(**public_task_data)

print("\nPublic task (sanitized):")
print(json.dumps(public_task.model_dump(), indent=2))

Public API fields: ['task_title', 'task_description', 'estimated_hours', 'priority']

Public task (sanitized):
{
  "task_title": "Implement user authentication",
  "task_description": "Add JWT-based authentication with refresh tokens",
  "estimated_hours": 8,
  "priority": "high"
}


## 10. Immutability and Thread Safety

Operable is frozen and hashable, enabling safe sharing across threads and caching.

In [30]:
# Operable is immutable (frozen dataclass)
try:
    user_op.name = "NewName"
except (AttributeError, TypeError) as e:
    print(f"✓ Immutability enforced: {type(e).__name__}")

try:
    user_op.__op_fields__ = ()
except (AttributeError, TypeError) as e:
    print(f"✓ Fields immutable: {type(e).__name__}")

✓ Immutability enforced: FrozenInstanceError
✓ Fields immutable: FrozenInstanceError


In [31]:
# Field ordering is preserved (deterministic serialization)
ordered_specs = [
    Spec(str, name="z_field"),
    Spec(str, name="a_field"),
    Spec(str, name="m_field"),
]

ordered_op = Operable(ordered_specs, name="Ordered")
field_order = [s.name for s in ordered_op.__op_fields__]

print(f"Field order (insertion order, not alphabetic): {field_order}")

# Verify it's a tuple (immutable and preserves order)
print(f"Type: {type(ordered_op.__op_fields__)}")

Field order (insertion order, not alphabetic): ['z_field', 'a_field', 'm_field']
Type: <class 'tuple'>


## 11. Multiple Models from Single Operable

DRY principle: Define schema once, generate multiple views.

In [32]:
# Complete user schema
complete_user_specs = [
    Spec(str, name="id"),
    Spec(str, name="username"),
    Spec(str, name="email"),
    Spec(str, name="password_hash"),
    Spec(str, name="api_key"),
    Spec(str, name="role"),
    Spec(bool, name="is_active"),
    Spec(str, name="created_at"),
]

complete_user_op = Operable(complete_user_specs, name="User")

# 1. Full internal model (all fields)
InternalUserModel = complete_user_op.create_model(adapter="pydantic", model_name="InternalUser")

# 2. Public API model (exclude sensitive fields)
PublicUserModel = complete_user_op.create_model(
    adapter="pydantic", model_name="PublicUser", exclude={"password_hash", "api_key"}
)

# 3. User creation request (client provides subset)
CreateUserModel = complete_user_op.create_model(
    adapter="pydantic", model_name="CreateUser", include={"username", "email", "role"}
)

# 4. User update request (partial update)
UpdateUserModel = complete_user_op.create_model(
    adapter="pydantic", model_name="UpdateUser", include={"email", "role", "is_active"}
)

print("Models generated from single Operable:")
print(f"1. InternalUser: {list(InternalUserModel.model_fields.keys())}")
print(f"2. PublicUser: {list(PublicUserModel.model_fields.keys())}")
print(f"3. CreateUser: {list(CreateUserModel.model_fields.keys())}")
print(f"4. UpdateUser: {list(UpdateUserModel.model_fields.keys())}")

Models generated from single Operable:
1. InternalUser: ['id', 'username', 'email', 'password_hash', 'api_key', 'role', 'is_active', 'created_at']
2. PublicUser: ['id', 'username', 'email', 'role', 'is_active', 'created_at']
3. CreateUser: ['email', 'role', 'username']
4. UpdateUser: ['email', 'is_active', 'role']


In [33]:
# Demonstrate usage
create_request = CreateUserModel(username="newuser", email="newuser@example.com", role="user")

print("\nCreate request (from client):")
print(json.dumps(create_request.model_dump(), indent=2))

# Server creates internal user (adds server-generated fields)
internal_user_data = {
    **create_request.model_dump(),
    "id": "usr_123",
    "password_hash": "$2b$12$...",
    "api_key": "sk_live_...",
    "is_active": True,
    "created_at": "2025-11-09T10:00:00Z",
}

internal_user = InternalUserModel(**internal_user_data)

print("\nInternal user (server):")
print(json.dumps(internal_user.model_dump(), indent=2))

# Return public view to client
public_data = internal_user.model_dump(include=set(PublicUserModel.model_fields.keys()))
public_user = PublicUserModel(**public_data)

print("\nPublic user (to client):")
print(json.dumps(public_user.model_dump(), indent=2))


Create request (from client):
{
  "email": "newuser@example.com",
  "role": "user",
  "username": "newuser"
}

Internal user (server):
{
  "id": "usr_123",
  "username": "newuser",
  "email": "newuser@example.com",
  "password_hash": "$2b$12$...",
  "api_key": "sk_live_...",
  "role": "user",
  "is_active": true,
  "created_at": "2025-11-09T10:00:00Z"
}

Public user (to client):
{
  "id": "usr_123",
  "username": "newuser",
  "email": "newuser@example.com",
  "role": "user",
  "is_active": true,
  "created_at": "2025-11-09T10:00:00Z"
}


## Summary Checklist

**Operable Basics:**
- ✅ Ordered, validated collection of Specs
- ✅ Enforces name uniqueness at construction (fail-fast)
- ✅ Accepts list or tuple input (converts to immutable tuple)
- ✅ Frozen dataclass (thread-safe, hashable)
- ✅ Preserves field insertion order (deterministic)

**Field Access:**
- ✅ `allowed()` returns set of field names (O(1) membership)
- ✅ `check_allowed()` validates names (raise or boolean mode)
- ✅ `get()` retrieves Spec by name (returns Unset if missing)
- ✅ `get_specs()` filters fields (include/exclude)

**Model Generation:**
- ✅ `create_model()` delegates to adapters (Pydantic, attrs, etc.)
- ✅ Supports field filtering (public/private views)
- ✅ Model name hierarchy: explicit > Operable.name > "DynamicModel"
- ✅ Framework-agnostic (Layer 2 of 3-layer architecture)

**Fuzzy Matching:**
- ✅ Handles LLM output variations (camelCase, spaces, capitalization)
- ✅ Strict mode (raise on unmatched) vs lenient (ignore unknown)
- ✅ Filters sentinel values (Unset, Undefined)

**Advanced Features:**
- ✅ Nested models (Spec with model types)
- ✅ Listable fields (list[T] via as_listable())
- ✅ Nullable fields (T | None via as_nullable())
- ✅ Validators transfer to generated models
- ✅ Defaults and default factories

**Patterns:**
- ✅ Single schema → Multiple views (DRY principle)
- ✅ Public/internal API models (exclude sensitive)
- ✅ Request/response models (include subsets)
- ✅ Structured LLM output parsing (fuzzy + validation)

**Design Principles:**
- ✅ Early validation (fail-fast at construction)
- ✅ Immutability (thread-safe, cacheable)
- ✅ Single responsibility (collection, not generation)
- ✅ Framework isolation (adapter pattern)
- ✅ Explicit over implicit (get_specs vs __iter__)

**Next Steps:**
- See `Spec` for individual field schema definition
- See `CustomParser` protocol for custom output format parsing
- See adapter tests for framework-specific features
- See `Element` for entity base classes using Operable